# Combine, standardise depth, and clean NetCDF data

## Imports

In [ ]:
import xarray as xr
import datetime
import cmath
from datetime import datetime
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib.dates as pldates
import numpy as np
import scipy.signal as sig
import scipy.integrate as integ
import scipy.interpolate as interp
import pandas as pd
import seawater as sea
from scipy.stats import chi2
from scipy.io import loadmat
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib notebook

## First file

Open the first NetCDF file, sort depth in ascending order, drop lat and long dimensions, get specific depth values for consecutive data to be mapped to, and if the initial file interpolate to those depths.

In [ ]:
# first file
ds = xr.open_dataset('../Data/SlopeAll/1.nc')     # open file
ds = ds.sortby('depth', ascending=True)           # sort depths ascending order

#ds = ds.drop_dims('longitude')                   # drop lat and long dimensions, if initial file
#ds = ds.drop_dims('latitude')
#depths = np.array(ds.depth.values,dtype=int)     # set specific depths for all data to map onto, if initial file
#depths = depths[depths >= 0]                     # keep only 'positive' depths
#ds = ds.interp(depth=depths,method='linear')     # interpolate initial data onto these depths, if initial file
print(depths)
print(ds)

In [ ]:
# plot to check data quality
fig,ax = plt.subplots(figsize=(9,4))
ax.pcolormesh(ds.time, -ds.depth, ds.u.T, rasterized=True, cmap='RdBu_r', vmin=-0.20, vmax=0.20)
plt.show()

## Second file

Open the next NetCDF file, sort depth in ascending order, drop lat and long dimensions, and interpolate over specific depths.

In [ ]:
# second file
ds0 = xr.open_dataset('../Data/SlopeAll/2.nc')   # open next file
ds0 = ds0.sortby('depth', ascending=True)        # sort depth in ascending order
ds0 = ds0.drop_dims('longitude')                 # drop lat and long dims
ds0 = ds0.drop_dims('latitude')
ds0 = ds0.interp(depth=depths,method='linear')   # interpolate onto specific depths

print(ds0)

In [ ]:
# plot to check data quality
fig,ax = plt.subplots(figsize=(9,4))
ax.pcolormesh(ds0.time, -ds0.depth, ds0.u.T, rasterized=True, cmap='RdBu_r', vmin=-0.20, vmax=0.20)
plt.show()

## Combine files

Combines first and next files. If necessary, remove bad data sections. Save file to new NetCDF.

In [ ]:
# combine files
ds1 = xr.combine_by_coords([ds0, ds])      # combine files

# start = 151500                           # to remove bad data
# end = 155000
# print(ds1.time[start:end])
# ds1.u[start:end,:] = np.nan
# ds1.v[start:end,:] = np.nan
# ds1.w[start:end,:] = np.nan

print(ds1)
#ds1.to_netcdf('../Data/SlopeAll/12.nc')   # save to new NetCDF file

In [ ]:
# plot to check data quality
fig,ax = plt.subplots(figsize=(9,4))
ax.pcolormesh(ds1.time, -ds1.depth, ds1.u.T, rasterized=True, cmap='RdBu_r', vmin=-0.20, vmax=0.20)
plt.show()